## Install Dependencies

In [ ]:
#!git clone https://github.com/GFDRR/caribbean-rooftop-classification.git
#!pip install -q segment-geospatial leafmap localtileserver

In [ ]:
#%cd caribbean-rooftop-classification/
#!git checkout add/tutorials

## Imports and Setup

In [ ]:
import os
import matplotlib.pyplot as plt
import rasterio as rio
import geopandas as gpd
import leafmap

from rasterio.plot import show
from shapely.geometry import box
from samgeo import SamGeo

## Download Image

In [ ]:
image_dir = "../data/rasters/drone/"
if not os.path.exists(image_dir):
    os.makedirs(image_dir)

filename = "drone_salisbury_cropped_DOM.tif"
image_file = os.path.join(image_dir, filename)

## Load Image

In [ ]:
image = rio.open(image_file)
data = gpd.GeoDataFrame({"id":1,"geometry":[box(*image.bounds)]}, crs=image.crs)
data = data.to_crs("EPSG:4326")
bbox = data.geometry.values[0]
centroid = bbox.centroid

In [ ]:
map = leafmap.Map(center=[centroid.y, centroid.x], zoom=20)
map.add_raster(image_file, layer_name="Image", zoom=20)
map

## Instantiate SamGeo

In [ ]:
sam = SamGeo(
    model_type="vit_h",
    checkpoint="sam_vit_h_4b8939.pth",
    device='cpu',
    sam_kwargs={"points_per_batch": 8},
)

## Generate Prediction

In [ ]:
mask = "segment.tif"
image_file = "../data/rasters/drone/drone_salisbury_cropped_DOM.tif"
sam.generate(image_file, mask, batch=True, foreground=True, mask_multiplier=255)